In [1]:
# ! pip install pyoso

## Setup

Load environment variables, import necessary libraries, and initialize the OSO client

In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()

OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

## Testing

Query the metrics table for all metric names containing '_funding_' and display them in alphabetical order

In [3]:
client.to_pandas("""
SELECT metric_name
FROM metrics_v0
WHERE metric_name LIKE '%_funding_%'
ORDER BY 1
""")

,metric_name
0,ARBITRUMFOUNDATION_funding_awarded_over_all_time
1,CLRFUND_funding_awarded_over_all_time
2,DAODROPS_funding_awarded_over_all_time
3,GITCOIN_DONATIONS_funding_awarded_biannually
4,GITCOIN_DONATIONS_funding_awarded_daily
5,GITCOIN_DONATIONS_funding_awarded_monthly
6,GITCOIN_DONATIONS_funding_awarded_over_all_time
7,GITCOIN_DONATIONS_funding_awarded_quarterly
8,GITCOIN_DONATIONS_funding_awarded_weekly
9,GITCOIN_DONATIONS_funding_awarded_yearly


## Aggregate funding metrics

### By source

We currently support CSV data uploads via [oss-funding](https://github.com/opensource-observer/oss-funding) and Gitcoin Grants. We also have Open Collective deposits, but they don't show up here (yet).

In [4]:
client.to_pandas("""
SELECT
  m.metric_name,
  SUM(km.amount) AS total_amount_in_usd
FROM key_metrics_by_project_v0 AS km
JOIN metrics_v0 AS m ON km.metric_id = m.metric_id
WHERE m.metric_name LIKE '%_funding_%'
GROUP BY 1
ORDER BY 2 DESC
""")

,metric_name,total_amount_in_usd
0,OPTIMISM_RETROFUNDING_funding_awarded_over_all...,138245667.349578
1,ARBITRUMFOUNDATION_funding_awarded_over_all_time,121538452.0
2,OPTIMISM_GOVGRANTS_funding_awarded_over_all_time,101425950.39
3,STELLAR_funding_awarded_over_all_time,33517284.98
4,GITCOIN_MATCHING_funding_awarded_over_all_time,12743948.169964
5,GITCOIN_DONATIONS_funding_awarded_over_all_time,10897861.066639
6,OCTANT_funding_awarded_over_all_time,3871958.50952
7,DAODROPS_funding_awarded_over_all_time,168634.0
8,CLRFUND_funding_awarded_over_all_time,55286.24013


### To projects

We can also see the largest project recipients with this query.

In [5]:
client.to_pandas("""
SELECT
  p.display_name AS project_display_name,
  SUM(km.amount) AS total_amount_in_usd
FROM key_metrics_by_project_v0 AS km
JOIN metrics_v0 AS m ON km.metric_id = m.metric_id
JOIN projects_v1 AS p ON km.project_id = p.project_id
WHERE m.metric_name LIKE '%_funding_awarded_over_all_time'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")

,project_display_name,total_amount_in_usd
0,GMX,21000000.0
1,MUX Protocol,10876479.0
2,Velodrome,10836736.04374
3,Gains Network,7909635.9075
4,Synthetix,7247628.074157
5,Camelot,5407500.0
6,Vertex Protocol,5250000.0
7,Radiant,4991077.0
8,Stargate Finance,4879172.708658
9,Perpetual Protocol,4787212.140718


### To projects from a specific source

In [6]:
client.to_pandas("""
SELECT
  p.display_name AS project_display_name,
  SUM(km.amount) AS total_amount_in_usd
FROM key_metrics_by_project_v0 AS km
JOIN metrics_v0 AS m ON km.metric_id = m.metric_id
JOIN projects_v1 AS p ON km.project_id = p.project_id
WHERE m.metric_name = 'GITCOIN_DONATIONS_funding_awarded_over_all_time'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")

,project_display_name,total_amount_in_usd
0,Gitcoin,1040187.136146
1,Revoke,748071.985745
2,DefiLlama,429924.507285
3,Hey,360529.24178
4,JediSwap,333277.670918
5,Dark Forest,332205.420888
6,ZigZag Exchange,210175.931949
7,rotki,159744.88374
8,ethers.js,149548.961252
9,Lighthouse by Sigma Prime,132816.547253


### To projects from a specific source and time frame

In [7]:
client.to_pandas("""
SELECT
  p.display_name AS project_display_name,
  SUM(tm.amount) AS total_amount_in_usd
FROM timeseries_metrics_by_project_v0 AS tm
JOIN metrics_v0 AS m ON tm.metric_id = m.metric_id
JOIN projects_v1 AS p ON tm.project_id = p.project_id
WHERE m.metric_name = 'GITCOIN_DONATIONS_funding_awarded_yearly'
AND tm.sample_date < DATE '2022-01-01'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")

,project_display_name,total_amount_in_usd
0,Gitcoin,797206.330376
1,Dark Forest,297517.115925
2,ZigZag Exchange,199746.433382
3,ethers.js,129500.966707
4,Prysm Ethereum Client,128522.705766
5,rotki,122666.927997
6,ZeroPool,116795.642612
7,Lighthouse by Sigma Prime,114759.839844
8,The Tor Project,110669.738113
9,Hardhat,110539.758225


### To/from funders (summary metrics)

In [8]:
df = client.to_pandas("""
SELECT
  p.project_id,
  p.display_name,
  REPLACE(m.metric_name, '_funding_awarded_over_all_time', '') AS funding_source,
  SUM(km.amount) AS total_amount_in_usd
FROM key_metrics_by_project_v0 AS km
JOIN metrics_v0 AS m ON km.metric_id = m.metric_id
JOIN projects_v1 AS p ON km.project_id = p.project_id
WHERE
  m.metric_name LIKE '%_funding_awarded_over_all_time'
  AND (km.amount >= 1000 AND km.amount < 1000000)
GROUP BY 1,2,3
ORDER BY 4 DESC
""")
df

,project_id,display_name,funding_source,total_amount_in_usd
0,dNDkpiWWFxWM+fz0JbPapZpz/TlsoYQCJevD3ICcAHU=,BOB,OPTIMISM_GOVGRANTS,997500.0
1,8ranj8ZjbHqnVkAyhSNS9HAKO0VN/Tb34jZdBROI788=,Mint Blockchain,OPTIMISM_GOVGRANTS,997500.0
2,qnQ1PFj4fkRd6WPxzt5EHNykmTn5G/nYVvWkWjs0h0g=,Revm,OPTIMISM_RETROFUNDING,982507.285439
3,ZtwRwweUfJxkM3rtVoIfvvOaqHRc7CGlSCAKE8VgmN0=,Aerodrome Finance,OPTIMISM_RETROFUNDING,958989.311521
4,lPxNsS/BATP5VXf5oTGLri62s8Um5J+Rm9nMBX+wFV4=,Lattice,OPTIMISM_RETROFUNDING,958503.015
...,...,...,...,...
2256,9FCd3zp3AhnThPkf6mxSbon88Co2iHmkcIfQAzdWbzc=,Dasy,GITCOIN_MATCHING,1010.41355
2257,R1dwI8DtnoQytJHlZmQ2puJgGCi+M0Fs0/7nr58a5/o=,DexKit,GITCOIN_DONATIONS,1007.749109
2258,Q8F6p4cN/9k6DpGmVYCcjZ67J1kV3bcJmPvDI5wwz2o=,dippixyz,GITCOIN_MATCHING,1001.623472
2259,uQO9q9SKY+pM9SqxIQXR83uXfbOy9jh9K77n91nU6QA=,RingsNetwork,GITCOIN_MATCHING,1000.425276


In [9]:
popular_projects = list(df.groupby('display_name')['funding_source'].nunique().sort_values(ascending=False).index[:10])
dff = df[df['display_name'].isin(popular_projects)].copy()
funder_names = {
    'OPTIMISM_RETROFUNDING': 'Optimism (Retro Funding)',
    'GITCOIN_MATCHING': 'Gitcoin (Matching)',
    'ARBITRUMFOUNDATION': 'Arbitrum STIP',
    'OPTIMISM_GOVGRANTS': 'Optimism (Gov Grants)',
    'OCTANT': 'Octant',     
    'GITCOIN_DONATIONS': 'Gitcoin (Donations)',           
    'STELLAR': 'Stellar Community Fund',              
    'DAODROPS': 'DAO Drops',               
    'CLRFUND': 'clr.fund'
}
dff['from_funder'] = dff['funding_source'].map(funder_names)
dff = dff[['from_funder', 'display_name', 'total_amount_in_usd']]
dff

,from_funder,display_name,total_amount_in_usd
26,Optimism (Retro Funding),rotki,773567.405
27,Optimism (Retro Funding),Giveth,763501.505973
60,Optimism (Retro Funding),growthepie,591839.955
107,Optimism (Retro Funding),The Metagovernance Project,445164.86
112,Gitcoin (Matching),rotki,437212.315468
165,Arbitrum STIP,Tally,350000.0
178,Optimism (Retro Funding),Tally,347827.48
187,Optimism (Retro Funding),Hypercerts,330436.12
233,Optimism (Retro Funding),Praise,277324.575
237,Optimism (Gov Grants),Tally,272000.0


## More granular analysis

### Gitcoin

Deep dive on Gitcoin grants to a specific project

In [10]:
client.to_pandas("""
  SELECT
    time,
    round_number,
    round_name,
    event_source,
    donor_address,
    amount_in_usd
  FROM int_events__gitcoin_funding
  WHERE gitcoin_group_project_name = 'revokecash'
  ORDER BY amount_in_usd DESC
  LIMIT 10
""")

,time,round_number,round_name,event_source,donor_address,amount_in_usd
0,2023-02-09 15:58:07.078,<NA>,Gitcoin Grants,GITCOIN_DONATIONS,0x386ea3171dcc9405311fd75b316cc2a87ecadeca,617893.575
1,2023-08-29 08:32:57.000,18,Web3 Open Source Software,GITCOIN_MATCHING,<NA>,15001.1375
2,2024-11-25 14:26:59.000,22,GG22 OSS - dApps and Apps,GITCOIN_MATCHING,<NA>,14984.28125
3,2024-05-31 14:35:02.000,20,dApps & Apps,GITCOIN_MATCHING,<NA>,14979.978125
4,2023-11-29 20:18:47.000,19,Web3 Open Source Software,GITCOIN_MATCHING,<NA>,14849.591509
5,2022-08-24 00:00:00.000,15,<NA>,GITCOIN_MATCHING,<NA>,12500.0
6,2024-08-26 15:19:00.000,21,GG21: Thriving Arbitrum Summer,GITCOIN_MATCHING,<NA>,9839.680095
7,2022-08-24 00:00:00.000,15,<NA>,GITCOIN_MATCHING,<NA>,7410.488854
8,2024-05-07 10:04:49.000,20,dApps & Apps,GITCOIN_DONATIONS,0xe2a26d5174b133abc4b338df1b07295f03a4c85e,1000.42865
9,2024-05-06 17:29:47.000,20,dApps & Apps,GITCOIN_DONATIONS,0x60a06b2eee871e349331143ef173ecefd7a8ce01,537.338562


## OSS Funding

Overview of specific funders and grant pools in oss-funding data

In [11]:
client.to_pandas("""
SELECT
  from_funder_name,
  COUNT(DISTINCT grant_pool_name) AS grant_pools,
  SUM(amount) AS amount_in_usd
FROM stg_ossd__current_funding
GROUP BY 1
ORDER BY 3 DESC
""")

,from_funder_name,grant_pools,amount_in_usd
0,optimism,15,275923907.037501
1,arbitrumfoundation,1,122850952.0
2,stellar,29,33589282.98
3,octant-golemfoundation,5,3965429.51329
4,dao-drops-dorgtech,1,250001.0
5,clrfund,1,83028.740386


## Funding flows

We can use this to construct a simple sankey diagram of funding flows

In [12]:
query = """
SELECT
  fp.project_id AS from_project_id,
  tp.project_id AS to_project_id,
  fp.display_name AS funder,
  tp.display_name AS project,
  SUM(e.amount) AS amount
FROM int_events_daily__funding AS e
JOIN artifacts_by_project_v1 AS fa
  ON e.from_artifact_id = fa.artifact_id
JOIN artifacts_by_project_v1 AS ta
  ON e.to_artifact_id = ta.artifact_id
JOIN projects_v1 AS fp
  ON fa.project_id = fp.project_id
JOIN projects_v1 AS tp
  ON ta.project_id = tp.project_id
GROUP BY 1,2,3,4
"""
df = client.to_pandas(query)
df.tail()

,from_project_id,to_project_id,funder,project,amount
2249,Qgbm336fY9862LN2Czg3UX04A3p7I/79Bv2M4D61DAI=,1rc31e43WnXR31VM//z41gY2UFQfbo2b8KCwLw0rY4k=,Gitcoin,rantomdotapp,6131.318222
2250,Qgbm336fY9862LN2Czg3UX04A3p7I/79Bv2M4D61DAI=,DRZVxhlH8Qia/HQsZTqIe+hZVn/I0wjhgSpHqcNn0RE=,Gitcoin,eggcess-tech,115.038641
2251,ta+k40iL4Aishk458IqTS4k+jcGiSCdYzpwaDPfKqss=,GFtPmpIWk2qUtklRVLF1k8VATmNH+3bHG4CA8iOfuxA=,Optimism,Yearn,1597827.48
2252,IW0dZPkUh9+SEfZy8z8Om57ZGyjR3ZmSWkeojs9qnbU=,rU5cF8xxSnfmw0tq9VCl33LJXFCXqQVzqAWnWN9iWnw=,Arbitrum Foundation,Synapse,3500000.0
2253,ta+k40iL4Aishk458IqTS4k+jcGiSCdYzpwaDPfKqss=,rU5cF8xxSnfmw0tq9VCl33LJXFCXqQVzqAWnWN9iWnw=,Optimism,Synapse,849991.175282
